In [ ]:
import numpy as np, torch, dill, time, os
from Ctubes.curve_parametrizations import trefoil_parameterization
from Ctubes.geometry_utils import rotate_about_axis, compute_cross_section_radii, regular_polygon, get_bisecting_plane_normals_with_symmetry
from Ctubes.plot_utils import plot_generatrix
from Ctubes.tubes import Directrix, Generatrix, CTube
from Ctubes.opt import CTubeOptimizationProblem
from Ctubes.misc_utils import get_pairings_exact, get_pairings_all
from Ctubes.path_utils import get_name, setup_paths

paths = setup_paths(get_name())

PI = np.pi
TORCH_DTYPE = torch.float64
torch.set_default_dtype(TORCH_DTYPE)
torch.set_printoptions(precision=4)

%load_ext autoreload
%autoreload 2

%matplotlib widget

# Figure 9: Trefoils

## #1: Curve Optimzation ($\varphi^* = 240$˚)

In [ ]:
# Define a path to output specific to the current test case
paths = setup_paths(get_name(), test_name="fig9_trefoil_240deg_curve")

In [ ]:
# Generatrix: regular N-gon
N = 3
tube_radius = 0.3
generatrix_2d = regular_polygon(N, tube_radius)

fig, ax = plot_generatrix(generatrix_2d)

In [ ]:
# Directrix: trefoil knot
closed_curve = True
K_per_base_curve = 5
n_fold_symm = 3
K = K_per_base_curve * n_fold_symm + 1
K_to_M_factor = 13
M = K_to_M_factor * (K - 1) + 1

ts_disc_curve = torch.linspace(0.0, 2.0 * PI, K)
cps_ref = trefoil_parameterization(ts_disc_curve)

In [ ]:
# Define symmetry transforms
z_axis = torch.tensor([0.0, 0.0, 1.0], dtype=TORCH_DTYPE)
symmetry_transforms = [
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(4*PI/3)),
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(8*PI/3)),
]

# Select one third of the curve
K = (K - 1) // n_fold_symm + 1
M = (M - 1) // n_fold_symm + 1
cps_ref = cps_ref[:K]

In [ ]:
# C-tube
directrix = Directrix(cps_ref, M, symmetry_transforms=symmetry_transforms)

X0 = directrix.X[0]
T0 = directrix.get_tangents()[0]
generatrix = Generatrix(generatrix_2d, X0, T0)

plane_normals = get_bisecting_plane_normals_with_symmetry(directrix)

tube = CTube(
    directrix, generatrix, plane_normals,
    symmetry_transforms=symmetry_transforms
)

In [ ]:
# Pickle C-Tube
pkl_file = os.path.join(paths["output_opt"], "tube_init.pkl")
dill.dump(tube, open(pkl_file, "wb"))

In [ ]:
fig, ax = tube.plot_3d()

In [ ]:
fig, ax = tube.plot_unrolled_strips()

### Optimization

In [ ]:
# Set up optimization problem

opt_weights = {
    'preserve_curve': 1.0 / directrix.aabb_diagonal_length() ** 2,
    'join_ends': 1e2 / generatrix.aabb_diagonal_length() ** 2,
}

objective_args = {
    'join_ends_pairings': get_pairings_all(N),
}

opt_prob = CTubeOptimizationProblem(
    tube, 
    opt_weights, 
    objective_args,
)

opt_prob.activate_cps(True)
opt_prob.activate_theta(False)
opt_prob.activate_apex_loc_func(False)
opt_prob.activate_plane_normals(False)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
from scipy.optimize import minimize
from Ctubes.opt import obj_and_grad

torch.autograd.set_detect_anomaly(False)

# Set up optimization configuration
opt_prob.configure_optimization_output(paths)

# Get initial parameters
params0 = opt_prob.get_params_numpy()

# Define objective and gradient function for SciPy
obj_and_grad_scipy = lambda params: obj_and_grad(params, opt_prob)

# Fix variables via double-sided bounds
fixed_indices = []
dofs = opt_prob.get_dof_indices_cps()
fixed_indices += dofs[:3] + dofs[-3:]  # fix endpoints
fixed_indices = list(set(fixed_indices))  # remove duplicates
fixed_indices = sorted(fixed_indices)

bounds = [(None, None)] * len(params0)
for idx in fixed_indices:
    bounds[idx] = (params0[idx], params0[idx])
print(f"Fixing {len(fixed_indices)} parameters.")

# Add initial state to history
opt_prob.add_objective_to_history()

In [ ]:
# Run optimization
start_time = time.time()
result = minimize(
    obj_and_grad_scipy, 
    params0, 
    jac=True, 
    method='L-BFGS-B',
    options={'ftol': 1.0e-10, 'gtol': 1.0e-5, 'disp': True, 'maxiter': 2000},
    bounds=bounds,
    callback=opt_prob.optimization_callback,
)
result.execution_time = time.time() - start_time

# Finalize optimization (save results, render videos, cleanup)
opt_prob.finalize_optimization(result)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
fig, ax = opt_prob.plot_objective_history()

In [ ]:
fig, ax = opt_prob.plot_3d()

In [ ]:
fig, ax = opt_prob.plot_unrolled_strips()

## #2: Curve Optimzation ($\varphi^* = 360$˚)

In [ ]:
# Load pickled C-Tube
paths_load = setup_paths(get_name(), test_name="fig9_trefoil_240deg_curve")

pkl_file = os.path.join(paths_load["output_opt"], "tube_init.pkl")
tube = dill.load(open(pkl_file, "rb"))

In [ ]:
paths = setup_paths(get_name(), test_name="fig9_trefoil_360deg_curve")

In [ ]:
fig, ax = tube.plot_3d()

In [ ]:
fig, ax = tube.plot_unrolled_strips()

### Optimization

In [ ]:
# Set up optimization problem

opt_weights = {
    'preserve_curve': 1.0 / directrix.aabb_diagonal_length() ** 2,
    'join_ends': 1e3 / generatrix.aabb_diagonal_length() ** 2,
}

objective_args = {
    'join_ends_pairings': get_pairings_exact(N),
}

opt_prob = CTubeOptimizationProblem(
    tube, 
    opt_weights,
    objective_args,
)

opt_prob.activate_cps(True)
opt_prob.activate_theta(False)
opt_prob.activate_apex_loc_func(False)
opt_prob.activate_plane_normals(False)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
from scipy.optimize import minimize
from Ctubes.opt import obj_and_grad

torch.autograd.set_detect_anomaly(False)

# Set up optimization configuration
opt_prob.configure_optimization_output(paths)

# Get initial parameters
params0 = opt_prob.get_params_numpy()

# Define objective and gradient function for SciPy
obj_and_grad_scipy = lambda params: obj_and_grad(params, opt_prob)

# Fix variables via double-sided bounds
fixed_indices = []
dofs = opt_prob.get_dof_indices_cps()
fixed_indices += dofs[:3] + dofs[-3:]  # fix endpoints
fixed_indices = list(set(fixed_indices))  # remove duplicates
fixed_indices = sorted(fixed_indices)

bounds = [(None, None)] * len(params0)
for idx in fixed_indices:
    bounds[idx] = (params0[idx], params0[idx])
print(f"Fixing {len(fixed_indices)} parameters.")

# Add initial state to history
opt_prob.add_objective_to_history()

In [ ]:
# Run optimization
start_time = time.time()
result = minimize(
    obj_and_grad_scipy, 
    params0, 
    jac=True, 
    method='L-BFGS-B',
    options={'ftol': 1.0e-10, 'gtol': 1.0e-5, 'disp': True, 'maxiter': 2000},
    bounds=bounds,
    callback=opt_prob.optimization_callback,
)
result.execution_time = time.time() - start_time

# Finalize optimization (save results, render videos, cleanup)
opt_prob.finalize_optimization(result)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
fig, ax = opt_prob.plot_objective_history()

In [ ]:
fig, ax = opt_prob.plot_3d()

In [ ]:
fig, ax = opt_prob.plot_unrolled_strips()

## #3: Plane Optimzation ($\varphi^* = 360$˚)

In [ ]:
# Load pickled C-Tube
paths_load = setup_paths(get_name(), test_name="fig9_trefoil_240deg_curve")

pkl_file = os.path.join(paths_load["output_opt"], "tube_init.pkl")
tube = dill.load(open(pkl_file, "rb"))

In [ ]:
paths = setup_paths(get_name(), test_name="fig9_trefoil_360deg_planes")

In [ ]:
fig, ax = tube.plot_3d()

In [ ]:
fig, ax = tube.plot_unrolled_strips()

### Optimization

In [ ]:
# Set up optimization problem

opt_weights = {
    'join_ends': 1.0 / generatrix.aabb_diagonal_length() ** 2,
}

objective_args = {
    'join_ends_pairings': get_pairings_exact(N),
}

opt_prob = CTubeOptimizationProblem(
    tube, 
    opt_weights, 
    objective_args,
)
opt_prob.activate_cps(False)
opt_prob.activate_theta(False)
opt_prob.activate_apex_loc_func(False)
opt_prob.activate_plane_normals(True)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
from scipy.optimize import minimize
from Ctubes.opt import obj_and_grad

torch.autograd.set_detect_anomaly(False)

# Set up optimization configuration
opt_prob.configure_optimization_output(paths)

# Get initial parameters
params0 = opt_prob.get_params_numpy()

# Define objective and gradient function for SciPy
obj_and_grad_scipy = lambda params: obj_and_grad(params, opt_prob)

# Fix variables via double-sided bounds
fixed_indices = []  # no fixed DOF by default

bounds = [(None, None)] * len(params0)
for idx in fixed_indices:
    bounds[idx] = (params0[idx], params0[idx])
print(f"Fixing {len(fixed_indices)} parameters.")

# Add initial state to history
opt_prob.add_objective_to_history()

In [ ]:
# Run optimization
start_time = time.time()
result = minimize(
    obj_and_grad_scipy, 
    params0, 
    jac=True, 
    method='L-BFGS-B',
    options={'ftol': 1.0e-10, 'gtol': 1.0e-5, 'disp': True, 'maxiter': 2000},
    bounds=bounds,
    callback=opt_prob.optimization_callback,
)
result.execution_time = time.time() - start_time

# Finalize optimization (save results, render videos, cleanup)
opt_prob.finalize_optimization(result)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
fig, ax = opt_prob.plot_objective_history()

In [ ]:
fig, ax = opt_prob.plot_3d()

In [ ]:
fig, ax = opt_prob.plot_unrolled_strips()

In [ ]:
pkl_file = os.path.join(paths["output_opt"], "tube.pkl")
dill.dump(tube, open(pkl_file, "wb"))

## #4: Plane + Apex-Locating Function Optimzation ($\varphi^* = 360$˚)

In [ ]:
# Load the result of the previous optimization round
paths_load = setup_paths(get_name(), test_name="fig9_trefoil_360deg_planes")

pkl_file = os.path.join(paths_load["output_opt"], "tube.pkl")
tube = dill.load(open(pkl_file, "rb"))

In [ ]:
paths = setup_paths(get_name(), test_name="fig9_trefoil_360deg_scale")

In [ ]:
fig, ax = tube.plot_3d()

In [ ]:
fig, ax = tube.plot_unrolled_strips()

### Optimization

In [ ]:
# Uniform target cross-section radius
target_cross_section_radii = torch.ones(M) * tube_radius

# Set init scale to match the target cross-section radius
rad_0 = compute_cross_section_radii(tube.get_polyline_with_symmetry(), tube.compute_vertices_with_symmetry())[0]
init_scale_match_target = target_cross_section_radii[0] / rad_0
tube.set_init_scale(init_scale_match_target)
tube.update_ctube_vertices()

In [ ]:
# Set up optimization problem

opt_weights = {
    'match_target_cross_section_radius': 5.25e1 / generatrix.aabb_diagonal_length() ** 2,
    'join_ends': 1e2 / generatrix.aabb_diagonal_length() ** 2,
    'smooth_plane_normal_diffs': 1e1,
}

objective_args = {
    'target_cross_section_radii': target_cross_section_radii,
    'join_ends_pairings': get_pairings_exact(N),
}

opt_prob = CTubeOptimizationProblem(
    tube, 
    opt_weights, 
    objective_args,
)
opt_prob.activate_cps(False)
opt_prob.activate_theta(False)
opt_prob.activate_apex_loc_func(True)
opt_prob.activate_plane_normals(True)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
from scipy.optimize import minimize
from Ctubes.opt import obj_and_grad

torch.autograd.set_detect_anomaly(False)

# Set up optimization configuration
opt_prob.configure_optimization_output(paths)

# Get initial parameters
params0 = opt_prob.get_params_numpy()

# Define objective and gradient function for SciPy
obj_and_grad_scipy = lambda params: obj_and_grad(params, opt_prob)

# Fix variables via double-sided bounds
fixed_indices = []  # no fixed DOF by default

bounds = [(None, None)] * len(params0)
for idx in fixed_indices:
    bounds[idx] = (params0[idx], params0[idx])
print(f"Fixing {len(fixed_indices)} parameters.")

# Add initial state to history
opt_prob.add_objective_to_history()

In [ ]:
# Run optimization
start_time = time.time()
result = minimize(
    obj_and_grad_scipy, 
    params0, 
    jac=True, 
    method='L-BFGS-B',
    options={'ftol': 1.0e-10, 'gtol': 1.0e-5, 'disp': True, 'maxiter': 1000},
    bounds=bounds,
    callback=opt_prob.optimization_callback,
)
result.execution_time = time.time() - start_time

# Finalize optimization (save results, render videos, cleanup)
opt_prob.finalize_optimization(result)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
fig, ax = opt_prob.plot_objective_history()

In [ ]:
fig, ax = opt_prob.plot_3d()

In [ ]:
fig, ax = opt_prob.plot_unrolled_strips()

In [ ]:
fig, ax = opt_prob.plot_cross_section_radii()